<a href="https://colab.research.google.com/github/Huxwell/ColabNNs/blob/main/export_coco_segmentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is based on : https://stackoverflow.com/users/14277626/eric-hofesmann , https://stackoverflow.com/questions/69845308/how-to-extract-foreground-objects-from-coco-dataset-or-open-images-v6-dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install fiftyone

# Visualisation

Not really usefull for us, but nice visualisation. It runs in a colab cell - I have zero idea how they got it working there!

You don't have to run it.


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

#
# Only the required images will be downloaded (if necessary).
# By default, only detections are loaded
#

dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["validation","train"],
    classes=["cat", "dog"],
    label_types=["segmentations"],
    max_samples=50,
)

# Visualize the dataset in the FiftyOne App
session = fo.launch_app(dataset)


export_dir = "/content/fiftyone-dataset-cats-dogs"

# The dataset or view to export
dataset_or_view = dataset

# Export the dataset without copying the media files
dataset_or_view.export(
    export_dir=export_dir,
    dataset_type=fo.types.FiftyOneDataset,
    export_media=True,
)

# Exports - the real, interesting code

In [ ]:
import os

import cv2
import numpy as np

def extract_classwise_instances(samples, output_dir, label_field, ext=".png"):
    print("Extracting object instances...")
    i = 0
    for sample in samples.iter_samples(progress=True):
        img = cv2.imread(sample.filepath)
        img_h,img_w,c = img.shape
        for det in sample[label_field].detections:
            i+=1
            if i % 10 == 0:
              print(i)
            mask = det.mask
            [x,y,w,h] = det.bounding_box
            x = int(x * img_w)
            y = int(y * img_h)
            h, w = mask.shape
            mask_img = img[y:y+h, x:x+w, :] 
            alpha = mask.astype(np.uint8)*255
            alpha = np.expand_dims(alpha, 2)
            mask_img = np.concatenate((mask_img, alpha), axis=2)
    
            label = det.label
            label_dir = os.path.join(output_dir, label)
            if not os.path.exists(label_dir):
                os.mkdir(label_dir)
            output_filepath = os.path.join(label_dir, det.id+ext)
            cv2.imwrite(output_filepath, mask_img)

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

dataset_name = "coco-image-example"
if dataset_name in fo.list_datasets():
    fo.delete_dataset(dataset_name)

label_field = "ground_truth"
classes = ["cat", "dog"]

dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["validation"],
    label_types=["segmentations"],
    classes=classes,
    # max_samples=90
    label_field=label_field,
    dataset_name=dataset_name,
)

view = dataset.filter_labels(label_field, F("label").is_in(classes))

output_dir = "/content/segmentation_from_coco_2017/val"
os.makedirs(output_dir, exist_ok=True)

extract_classwise_instances(view, output_dir, label_field)

In [ ]:
dataset_name = "coco-dogs-cats-segmentation-train"


dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train"],
    label_types=["segmentations"],
    classes=classes,
    # max_samples=90
    label_field=label_field,
    dataset_name=dataset_name,
)

view = dataset.filter_labels(label_field, F("label").is_in(classes))

output_dir = "/content/segmentation_from_coco_2017/train"
os.makedirs(output_dir, exist_ok=True)

extract_classwise_instances(view, output_dir, label_field)

In [ ]:
!ls /content/segmentation_from_coco_2017/val/dog | wc -l 
!ls /content/segmentation_from_coco_2017/val/cat | wc -l 
!ls /content/segmentation_from_coco_2017/train/dog | wc -l 
!ls /content/segmentation_from_coco_2017/train/cat | wc -l 

In [ ]:
# !cp -r /content/segmentation_from_coco_2017/ /content/drive/MyDrive/segmentation_from_coco_2017/

In [ ]:
!rsync -rah --info=progress2 /content/segmentation_from_coco_2017/ /content/drive/MyDrive/segmentation_from_coco_2017/

In [ ]:
# !rm -rf /content/segmentation_from_coco_2017/train